In [2]:
import os
import pandas as pd
from IPython.display import display, HTML
import re

#path = "c:\\data\\"
path = "/Users/jeff/temp/dissertation"


filename_read = os.path.join(path,"workload.csv")
df = pd.read_csv(filename_read)


# Experiment 1

In [3]:
import re

s = 'neural-r:sqrt-y0|rmse'
result = re.search('\:(.*)\|', s).group(1)
print(result.group(1))

AttributeError: 'str' object has no attribute 'group'

In [4]:
lookup = {'diff-y0' : 'Difference',
'log-y0' : 'Log',
'poly-y0' : 'Polynomial',
'poly2-y0' : 'Polynomial2',
'pow-y0' : 'Power',
'ratio-y0' : 'Ratio',
'ratio_diff-y0' : 'Ratio Difference',
'ratio_poly-y0' : 'Ratio Polynomial',
'ratio_poly2-y0' : 'Ratio Polynomial2',
'sqrt-y0' : 'Square Root'}


ex1 = df[df.name == 'experiment-1'][['dataset','algorithm','predictors','cycle','result','result-raw']]
f = {'result':['min','mean','std']}
ex1_summary = ex1.groupby(['algorithm']).agg(f)

ex1_summary.columns = [x[1] for x in ex1_summary.columns.values]
ex1_summary.insert(0,'algorithm', ex1_summary.index)
ex1_summary.insert(1, 'name' ,[re.search('\:(.*)\|', x).group(1) for x in ex1_summary.algorithm])
ex1_summary.insert(0, 'set' , [('gp' if 'gp' in x else 'neural') for x in ex1_summary.algorithm])
ex1_summary = ex1_summary[ex1_summary.name.isin(lookup.keys())]
ex1_summary.insert(1, 'expression' ,[lookup[x] for x in ex1_summary.name])
ex1_summary.sort_values(by=['expression'],inplace=True)
ex1_summary.index = range(len(ex1_summary))
ex1_summary.insert(0, 'num', ["{}-{}".format(1,x+1) for x in range(len(ex1_summary))])



In [5]:
# Experiment 1 neural results
ex1_summary_neural = ex1_summary[ex1_summary['set'] == 'neural'][['num','expression','min','mean','std']]
display(ex1_summary_neural)
ex1_summary_neural.to_csv(os.path.join(path,"diss_ex1_neural.csv"),index=False)

,num,expression,min,mean,std
1,1-2,Difference,0.007058,0.652730,0.360942
3,1-4,Log,0.210470,0.437343,0.310659
5,1-6,Polynomial,3.500132,3.500497,0.000336
7,1-8,Polynomial2,1.137178,1.137338,0.000147
9,1-10,Power,0.019174,0.231390,0.121577
11,1-12,Ratio,27.421797,27.453531,0.029588
13,1-14,Ratio Difference,22.982698,22.983074,0.000515
14,1-15,Ratio Polynomial,10.141962,10.232286,0.050493
16,1-17,Ratio Polynomial2,0.080145,0.164404,0.053830
19,1-20,Square Root,0.016608,0.152178,0.111331


In [6]:
# Experiment 1 GP results
ex1_summary_gp = ex1_summary[ex1_summary['set'] == 'gp'][['num','expression','min','mean','std']]
display(ex1_summary_gp)
ex1_summary_gp.to_csv(os.path.join(path,"diss_ex1_gp.csv"),index=False)

,num,expression,min,mean,std
0,1-1,Difference,0.000000,0.114616,0.256290
2,1-3,Log,0.071892,0.650784,0.404413
4,1-5,Polynomial,1.078862,1.567133,0.676108
6,1-7,Polynomial2,0.931649,1.236137,0.174696
8,1-9,Power,0.000000,0.059004,0.131937
10,1-11,Ratio,0.000000,5.496133,12.289726
12,1-13,Ratio Difference,22.982817,22.985711,0.003232
15,1-16,Ratio Polynomial,7.816065,9.282896,1.333794
17,1-18,Ratio Polynomial2,0.105438,0.187922,0.046158
18,1-19,Square Root,0.000468,0.043534,0.041200


# Experiment 2

In [7]:
ex2 = df[df.name == 'experiment-2'][['dataset','algorithm','predictors','cycle','result','result-raw']]
#ex2 = ex2[ex2.dataset != 'feature_eng.csv']

f = {'result-raw':['min','mean','std']}
ex2_summary = ex2.groupby(['algorithm','dataset']).agg(f)

ex2_summary.columns = [x[1] for x in ex2_summary.columns.values]
ex2_summary.insert(0,'algorithm', [x[0] for x in ex2_summary.index])
ex2_summary.insert(0,'dataset', [x[1] for x in ex2_summary.index])
ex2_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex2_summary.dataset])
ex2_summary.insert(0, 'set' , [('gp' if 'gp' in x else 'neural') for x in ex2_summary.algorithm])
ex2_summary.index = range(len(ex2_summary))
ex2_summary.sort_values(by=['set','name'],inplace=True)

ex2_summary_neural = ex2_summary[ex2_summary['set'] == 'neural'][['name','min','mean','std']]
ex2_summary_neural.columns = ['name','neural_min','neural_mean','neural_std']
ex2_summary_gp = ex2_summary[ex2_summary['set'] == 'gp'][['name','min','mean','std']]
ex2_summary_gp.columns = ['name','gp_min','gp_mean','gp_std']

#display(ex2_summary_neural)
#display(ex2_summary_gp)
display(pd.merge(ex2_summary_neural,ex2_summary_gp,how="left"),on=['name'])

,name,neural_min,neural_mean,neural_std,gp_min,gp_mean,gp_std
0,abalone,2.114894,2.572675,0.487028,1.140881,1.321847e+00,1.287754e-01
1,auto-mpg,2.244318,4.398705,2.902057,2.733030,3.790242e+00,7.476644e-01
2,bupa,0.411781,0.431751,0.014281,0.340094,3.637265e-01,2.017010e-02
3,covtype,0.359534,0.400625,0.030079,NaN,NaN,NaN
4,crx,0.332178,0.353303,0.021538,0.000000,0.000000e+00,0.000000e+00
5,forestfires,0.057975,0.062246,0.002668,NaN,NaN,NaN
6,glass,0.317631,0.327137,0.011926,NaN,NaN,NaN
7,heart_cleveland,0.319744,0.335615,0.018773,NaN,NaN,NaN
8,hepatitis,0.446050,0.454582,0.016504,0.000000,2.212727e-10,4.424852e-10
9,horse-colic,0.337819,0.362577,0.014049,NaN,NaN,NaN


# Experiment 3

In [8]:
ex3 = df[df.name == 'experiment-3'][['dataset','algorithm','predictors','cycle','result','result-raw']]

f = {'result-raw':['min','mean','std']}
ex3_summary = ex3.groupby(['dataset']).agg(f)

ex3_summary.columns = [x[1] for x in ex3_summary.columns.values]
ex3_summary.insert(0,'dataset', ex3_summary.index)
ex3_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex3_summary.dataset])
ex3_summary.index = range(len(ex3_summary))
ex3_summary.drop('dataset', axis=1, inplace=True)
ex3_summary.columns = ['name','ensemble_min','ensemble_mean','ensemble_std']

ex3_summary = pd.merge(ex3_summary,ex2_summary_neural,how="left",on=['name'])
ex3_summary = pd.merge(ex3_summary,ex2_summary_gp,how="left",on=['name'])
ex3_summary.drop(['neural_std','neural_min','gp_std','gp_min'], axis=1, inplace=True)

display(ex3_summary)

,name,ensemble_min,ensemble_mean,ensemble_std,neural_mean,gp_mean
0,abalone,2.415381,2.729983,0.306660,2.572675,1.321847e+00
1,auto-mpg,2.367463,2.520249,0.177967,4.398705,3.790242e+00
2,bupa,0.428034,0.444665,0.015502,0.431751,3.637265e-01
3,crx,0.326418,0.356366,0.023253,0.353303,0.000000e+00
4,hepatitis,0.441576,0.473846,0.027822,0.454582,2.212727e-10
5,pima-indians-diabetes,0.391882,0.401453,0.010405,0.413272,2.864604e-01
6,wcbreast_wdbc,0.080417,0.089454,0.013049,0.095292,2.300154e-01
7,wcbreast_wpbc,0.352044,0.357472,0.006107,0.382045,7.915624e-02


# Experiment 6

In [9]:
ex6 = df[df.name == 'experiment-6'][['dataset','algorithm','predictors','cycle','result','result-raw']]

f = {'result-raw':['min','mean','std']}
ex6_summary = ex6.groupby(['algorithm','dataset']).agg(f)

ex6_summary.columns = [x[1] for x in ex6_summary.columns.values]
ex6_summary.insert(0,'algorithm', [x[0] for x in ex6_summary.index])
ex6_summary.insert(0,'dataset', [x[1] for x in ex6_summary.index])
ex6_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex6_summary.dataset])
ex6_summary.index = range(len(ex6_summary))
ex6_summary.drop(['dataset','algorithm'], axis=1, inplace=True)
ex6_summary.columns = ['name','auto_min','auto_mean','auto_std']
ex6_summary = pd.merge(ex6_summary,ex3_summary,how="left",on=['name'])
ex6_summary = pd.merge(ex6_summary,ex2_summary_neural,how="left",on=['name'])
ex6_summary.neural_mean_x.fillna(ex6_summary.neural_mean_y,inplace=True)
ex6_summary.drop(['auto_min','auto_std','ensemble_min','ensemble_std','neural_mean_y','neural_min','neural_std'], 
                 axis=1, inplace=True)
ex6_summary.rename(columns={'neural_mean_x':'neural_mean'},inplace=True)
ex6_summary['best_non_fe'] = ex6_summary[['ensemble_mean','neural_mean','gp_mean']].min(axis=1,skipna=True)
ex6_summary['diff'] = ex6_summary.auto_mean - ex6_summary.best_non_fe
ex6_summary['pct'] = (ex6_summary.auto_mean - ex6_summary.best_non_fe) / ex6_summary.best_non_fe

ex6_summary.sort_values(by=['name'],inplace=True)
ex6_summary.index = range(len(ex6_summary))



display(ex6_summary)

,name,auto_mean,ensemble_mean,neural_mean,gp_mean,best_non_fe,diff,pct
0,abalone,4.307904,2.729983,2.572675,1.321847e+00,1.321847e+00,2.986056,2.259002e+00
1,auto-mpg,2.454458,2.520249,4.398705,3.790242e+00,2.520249e+00,-0.065791,-2.610496e-02
2,bupa,0.495160,0.444665,0.431751,3.637265e-01,3.637265e-01,0.131434,3.613531e-01
3,covtype,0.408135,NaN,0.400625,NaN,4.006247e-01,0.007511,1.874721e-02
4,crx,0.374440,0.356366,0.353303,0.000000e+00,0.000000e+00,0.374440,inf
5,forestfires,0.061719,NaN,0.062246,NaN,6.224644e-02,-0.000528,-8.477116e-03
6,glass,0.342102,NaN,0.327137,NaN,3.271373e-01,0.014965,4.574413e-02
7,heart_cleveland,0.348581,NaN,0.335615,NaN,3.356152e-01,0.012966,3.863213e-02
8,hepatitis,0.463375,0.473846,0.454582,2.212727e-10,2.212727e-10,0.463375,2.094134e+09
9,horse-colic,0.361303,NaN,0.362577,NaN,3.625768e-01,-0.001274,-3.514404e-03
